In [99]:
import numpy as np

# Constants
c = 3e8 # m/s       speed of light
Re = 6378e3 # m     radius of earth


# Variables
Uplink_frequency = 435e6 # Hz
Downlink_frequency = 435e6 # Hz

satellite_altitude = 400e3 # m

Cubesat_transmitter_power = -10 # dBW
Cubesat_antenna_gain = 2.15 # dBi
Cubesat_line_loss = 0 # dB   #line losses

Ground_station_transmitter_power = 10 # dBW
Ground_station_antenna_gain = 15 # dBi
Ground_station_line_loss = 2 # dB   #line losses

Uplink_atmospheric_loss = 0.5 # dB
Downlink_atmospheric_loss = 0.5 # dB
polarization_loss = 3 # dB
Ground_station_pointing_loss = 0 # dB

Effective_noise_temperature_cube = 470 # K
Effective_noise_temperature_gs = 1000 # K

bandwidth = 12e2 # Hz   bandwidth of the signal
EB_NO_req = 10 # dB     required energy per bit to noise density ratio


EIRP_cubesat = Cubesat_transmitter_power + Cubesat_antenna_gain # dB
EIRP_station = Ground_station_transmitter_power + Ground_station_antenna_gain # dB

In [100]:
def calculate_distance_to_satellite(angle, degrees=False, unit='m'):
    if degrees: angle = np.deg2rad(angle)
    if(unit == 'km'): return ((Re*np.sin(angle)*-1) + np.sqrt((Re * np.sin(angle))**2+ 2*Re*satellite_altitude + satellite_altitude**2))/1000
    else: return (Re*np.sin(angle)*-1) + np.sqrt((Re * np.sin(angle))**2+ 2*Re*satellite_altitude + satellite_altitude**2)

def free_space_path_loss(frequency, distance):
    return 20 * np.log10(frequency) + 20 * np.log10(distance) + 20 * np.log10(4 * np.pi / c )

def dipole_radiation_pattern_db(theta, degrees=False):
    if degrees: theta = np.deg2rad(theta)
    return 10*np.log10(((np.cos((np.pi/2)*np.cos(theta))) /(np.sin(theta)))**2)+2.15

def calculate_depiontning_loss(epsilon, degrees=False):
    if degrees: epsilon = np.deg2rad(epsilon)
    n = np.arcsin((Re*np.cos(epsilon))/(Re+satellite_altitude))
    return dipole_radiation_pattern_db((np.pi/2)-n)

def isotropic_received_power(EIRP, fspl, polarisation_loss, atmospheric_loss, depointing_loss):
    return EIRP - fspl - polarisation_loss - atmospheric_loss - depointing_loss

def figure_of_merit(gain, losses, temperature):
    return gain - losses - 10*np.log10(temperature)

def noise_power(temperature, bandwidth):
    return 10*np.log10(1.38e-23*temperature*bandwidth)


In [101]:

angle = 90
d = calculate_distance_to_satellite(angle, True)
fspl = free_space_path_loss(Downlink_frequency, d)
iso_pwr = isotropic_received_power(EIRP_cubesat, fspl, polarization_loss, Downlink_atmospheric_loss, -1*(calculate_depiontning_loss(angle, True)-2.15))
received_power = iso_pwr - Ground_station_pointing_loss + Ground_station_antenna_gain - Ground_station_line_loss

print("Downlink")
print("Elevation angle(°): ", angle)
print("Distance to satellite (km): ", calculate_distance_to_satellite(angle, True, 'km'))
print("Frequency (MHz): " , Downlink_frequency/1e6)
print("---Cubesat---")
print("Transmitter power (dBW): ", Cubesat_transmitter_power)
print("Antenna gain (dBi): ", Cubesat_antenna_gain)
print("Cubesat EIRP (dBW): ", EIRP_cubesat)
print("---Channel---")
print("Cubesat pointing loss (dB): ", -1*(calculate_depiontning_loss(angle, True)-2.15))
print("polarization loss (dB): ", polarization_loss)
print("Atmospheric loss (dB): ", Downlink_atmospheric_loss)
print("Free space path loss (dB): ", fspl)
print("Isotropic received power (dBW): ", iso_pwr)
print("---Ground station---")
print("Ground station pointing loss (dB): ", Ground_station_pointing_loss)
print("Antenna gain (dBi): ", Ground_station_antenna_gain)
print("Ground station line loss (dB): ", Ground_station_line_loss)
print("Effective_noise_temperature_gs (K): ", Effective_noise_temperature_gs)
print("Figure of merit (dB/K): ", figure_of_merit(Ground_station_antenna_gain, Ground_station_line_loss, Effective_noise_temperature_gs))
print("Received power (dBW): ", received_power)
print("Power noise temperature (dBW): ", noise_power(Effective_noise_temperature_gs, bandwidth))
print('Signal to noise ratio (dB): ', received_power - noise_power(Effective_noise_temperature_gs, bandwidth))
print("SNR requirement (dB): ", EB_NO_req)
print("Margin (dB): ", received_power - noise_power(Effective_noise_temperature_gs, bandwidth) - EB_NO_req)

Downlink
Elevation angle(°):  90
Distance to satellite (km):  400.0
Frequency (MHz):  435.0
---Cubesat---
Transmitter power (dBW):  -10
Antenna gain (dBi):  2.15
Cubesat EIRP (dBW):  -7.85
---Channel---
Cubesat pointing loss (dB):  -0.0
polarization loss (dB):  3
Atmospheric loss (dB):  0.5
Free space path loss (dB):  137.25275715170068
Isotropic received power (dBW):  -148.60275715170067
---Ground station---
Ground station pointing loss (dB):  0
Antenna gain (dBi):  15
Ground station line loss (dB):  2
Effective_noise_temperature_gs (K):  1000
Figure of merit (dB/K):  -17.0
Received power (dBW):  -135.60275715170067
Power noise temperature (dBW):  -167.80939667551138
Signal to noise ratio (dB):  32.20663952381071
SNR requirement (dB):  10
Margin (dB):  22.206639523810708


In [102]:
angle = 90
d = calculate_distance_to_satellite(angle, True)
fspl = free_space_path_loss(Uplink_frequency, d)
iso_pwr = isotropic_received_power(EIRP_cubesat, fspl, polarization_loss, Uplink_atmospheric_loss, Ground_station_pointing_loss)
cube_pointing_loss = (calculate_depiontning_loss(angle, True)-2.15)*-1
received_power = iso_pwr - cube_pointing_loss + Cubesat_antenna_gain - Cubesat_line_loss

print("Uplink")
print("Elevation angle(°): ", angle)
print("Distance to satellite (km): ", calculate_distance_to_satellite(angle, True, 'km'))
print("Frequency (MHz): " , Uplink_frequency/1e6)
print("---Ground station---")
print("Transmitter power (dBW): ", Ground_station_transmitter_power)
print("Transmission lines loss (dB): ", Ground_station_line_loss)
print("Antenna gain (dBi): ", Ground_station_antenna_gain)
print("Ground station EIRP (dBW): ", EIRP_station)
print("---Channel---")
print("Ground Station pointing loss (dB): ", Ground_station_pointing_loss)
print("polarization loss (dB): ", polarization_loss)
print("Atmospheric loss (dB): ", Uplink_atmospheric_loss)
print("Free space path loss (dB): ", fspl)
print("Isotropic received power (dBW): ", iso_pwr)
print("---Cubesat---")
print("Cubesat pointing loss (dB): ", cube_pointing_loss)
print("Antenna gain (dBi): ", Cubesat_antenna_gain)
print("Ground station line loss (dB): ", Cubesat_line_loss)
print("Effective_noise_temperature_gs (K): ", Effective_noise_temperature_cube)
print("Figure of merit (dB/K): ", figure_of_merit(Cubesat_antenna_gain, Cubesat_line_loss, Effective_noise_temperature_cube))
print("Received power (dBW): ", received_power)
print("Power noise temperature (dBW): ", noise_power(Effective_noise_temperature_cube, bandwidth))
print('Signal to noise ratio (dB): ', received_power - noise_power(Effective_noise_temperature_cube, bandwidth))
print("SNR requirement (dB): ", EB_NO_req)
print("Margin (dB): ", received_power - noise_power(Effective_noise_temperature_cube, bandwidth) - EB_NO_req)

Uplink
Elevation angle(°):  90
Distance to satellite (km):  400.0
Frequency (MHz):  435.0
---Ground station---
Transmitter power (dBW):  10
Transmission lines loss (dB):  2
Antenna gain (dBi):  15
Ground station EIRP (dBW):  25
---Channel---
Ground Station pointing loss (dB):  0
polarization loss (dB):  3
Atmospheric loss (dB):  0.5
Free space path loss (dB):  137.25275715170068
Isotropic received power (dBW):  -148.60275715170067
---Cubesat---
Cubesat pointing loss (dB):  -0.0
Antenna gain (dBi):  2.15
Ground station line loss (dB):  0
Effective_noise_temperature_gs (K):  470
Figure of merit (dB/K):  -24.570978579357178
Received power (dBW):  -146.45275715170067
Power noise temperature (dBW):  -171.08841809615421
Signal to noise ratio (dB):  24.635660944453548
SNR requirement (dB):  10
Margin (dB):  14.635660944453548
